In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

inporting data
### improting data :devil:


In [46]:
training_data = datasets.FashionMNIST(
    root= "data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    transform=ToTensor(),
    download=True
)

In [47]:
batch_size =  64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
for X,y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [48]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu" 
)
print("using", device,"device")

using cuda device


In [49]:
class NN(nn.Module ):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()     # flatten turns torch.Size([64, 1, 28, 28]) to [64,28*28]
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NN().to(device)
print(model)

NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [50]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
# print(model.parameters())

In [51]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct=0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item() #item converts tensor to number
            # print(pred)       # has some interesting results showing how argmax
            correct += (pred.argmax(1) ==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

In [52]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303862  [   64/60000]
loss: 2.292822  [ 6464/60000]
loss: 2.266240  [12864/60000]
loss: 2.258803  [19264/60000]
loss: 2.242189  [25664/60000]
loss: 2.220242  [32064/60000]
loss: 2.228049  [38464/60000]
loss: 2.200084  [44864/60000]
loss: 2.180411  [51264/60000]
loss: 2.156468  [57664/60000]
Test Error: 
 Accuracy: 47.7%, Avg loss: 2.149305

Epoch 2
-------------------------------
loss: 2.157489  [   64/60000]
loss: 2.158443  [ 6464/60000]
loss: 2.091334  [12864/60000]
loss: 2.106039  [19264/60000]
loss: 2.051613  [25664/60000]
loss: 1.998050  [32064/60000]
loss: 2.023937  [38464/60000]
loss: 1.948196  [44864/60000]
loss: 1.935234  [51264/60000]
loss: 1.873705  [57664/60000]
Test Error: 
 Accuracy: 57.0%, Avg loss: 1.871392

Epoch 3
-------------------------------
loss: 1.897854  [   64/60000]
loss: 1.887982  [ 6464/60000]
loss: 1.754279  [12864/60000]
loss: 1.795481  [19264/60000]
loss: 1.676555  [25664/60000]
loss: 1.636167  [32064/60000